In [18]:
import random
import pandas as pd
import numpy as np

In [19]:
def get_params():
  n = int(input('Введите количество устройств n'))
  m = int(input('Введите количество задач m'))
  r1, r2 = [int(x) for x in input('Введите левую и правую границы генерации r1 r2').split(' ')]
  k = int(input('Введите количество повторов k'))
  n_ind = int(input('Введите количество особей в поколении Ind'))
  p_cross = float(input('Введите вероятность кроссовера Pk'))
  p_mut = float(input('Введите вероятность мутации Pm'))
  model_type = int(input('Введите модель генетического алгоритма (1. Голберг 2. Холланд)'))
  
  if model_type == 1:
    return n, np.random.randint(r1, r2, m).tolist(), p_mut, p_cross, n_ind,  k, model_type
  else:
    return n, np.random.randint(r1, r2, (m, n)).tolist(), p_mut, p_cross, n_ind, k, model_type

In [20]:
class Individual():

  def __init__(self, n, task_vector, task_matrix, p_mut, p_cross, num, model):
      self.model = model
      self.n = n
      self.x_gens = task_vector
      self.full_x_gens = task_matrix
      self.x_holland = []
      self.y_gens = self.get_y_gens()
      self.mut_prop = [*[1 for i in range(int(p_mut * 100))], *[0 for i in range(100 - int(p_mut * 100))]]
      self.cross_prop = [*[1 for i in range(int(p_cross * 100))], *[0 for i in range(100 - int(p_cross * 100))]]
      self.p_mut = p_mut
      self.p_cross = p_cross
      self.phenotype = self.get_pheno()
      self.num = num

  def get_y_gens(self):
    interval_size = 256 // self.n
    ranges = []
    phenos = {}

    start = 0
    for i in range(1, self.n+1):
        end = 255 - (self.n - i) * interval_size
        ranges.append((start, end))
        phenos[f'{start}...{end}'] = []
        start = end + 1
        
    rows, cols = self.full_x_gens.shape
    
    self.y_gens = []
    
    for row in range(rows):
        curr_indx = np.random.randint(0, self.n)
        while self.full_x_gens[row, curr_indx] == -1:
            curr_indx = np.random.randint(0, self.n)
        self.y_gens.append(np.random.randint(ranges[curr_indx][0], ranges[curr_indx][1]))
        
    return self.y_gens
          
  def print_info(self):
        print(f"Генотип -->")
        if self.model == 1:
            print("\t" + " ".join(map(str, self.x_gens)))
        else:
            print("\t" + " ".join(map(str, self.x_holland)))
        print("\t" + " ".join(map(str, self.y_gens)))
        self.print_pheno()
        print("\n")
  
  def print_matrix(self):
    print(pd.DataFrame(self.full_x_gens))
        
  def get_pheno(self):
    interval_size = 256 // self.n
    ranges = []
    phenos = {}

    start = 0
    for i in range(1, self.n+1):
        end = 255 - (self.n - i) * interval_size
        ranges.append((start, end))
        phenos[f'{start}...{end}'] = []
        start = end + 1
    
    self.x_holland = []
    for gen_num in range(len(self.y_gens)):
        for range_num in range(len(ranges)):
            r = ranges[range_num]
            if r[0] <= self.y_gens[gen_num] < r[1]:
                if self.model == 1:
                    phenos[f'{r[0]}...{r[1]}'].append(self.x_gens[gen_num])
                else:
                    phenos[f'{r[0]}...{r[1]}'].append(self.x_gens[gen_num][range_num])
                    self.x_holland.append(self.x_gens[gen_num][range_num])
                
    max_feno_num = np.argmax([sum(values) for values in phenos.values()])
    max_feno = list(phenos.values())[max_feno_num]
    
    return {
        'ranges': ranges,
        'phenos': phenos,
        'max_feno': max_feno,
        'max_feno_num': max_feno_num
    }
    
  def update_pheno(self):
    self.phenotype = self.get_pheno()

  def print_pheno(self):
    print(f"Фенотип:")
    for num, (pheno, values) in enumerate(self.phenotype['phenos'].items()):
        sum_values = sum(values)
        max_indicator = "<-- MAX" if num == self.phenotype['max_feno_num'] else ""
        print(f"{num+1}) {pheno} : {values} | Сумма: {sum_values} {max_indicator}")

  def mutation(self):
    if random.random() < (self.mut_prop.count(1) / 100):
        print(f"Произошла мутация (её шанс был {self.p_mut})", end='\n')
        
        interval_size = 256 // self.n
        ranges = []
        phenos = {}

        start = 0
        for i in range(1, self.n + 1):
            end = 255 - (self.n - i) * interval_size
            ranges.append((start, end))
            phenos[f'{start}...{end}'] = []
            start = end + 1

        original_gene_index = random.randint(0, len(self.y_gens) - 1)
        original_gene = self.y_gens[original_gene_index]
        mutation_attempts = 3
        success = False

        for _ in range(mutation_attempts):
            random_gene_index = original_gene_index
            random_gene = original_gene

            if self.model == 1:
                while random_gene.bit_length() == 0:
                    random_gene_index = random.randint(0, len(self.y_gens) - 1)
                    random_gene = self.y_gens[random_gene_index]

                bit_length = random_gene.bit_length()
                random_bit_position = random.randint(0, bit_length - 1)
                mask = 1 << random_bit_position
                mutated_gene = random_gene ^ mask

            else:
                while random_gene.bit_length() < 2:
                    random_gene_index = random.randint(0, len(self.y_gens) - 1)
                    random_gene = self.y_gens[random_gene_index]

                bit_length = random_gene.bit_length()
                random_bit_positions = random.sample(range(bit_length), 2)
                mask1 = 1 << random_bit_positions[0]
                mask2 = 1 << random_bit_positions[1]
                mutated_gene = random_gene ^ mask1 ^ mask2

            interval_number = next(i for i, (start, end) in enumerate(ranges) if start <= mutated_gene <= end)
            value = self.full_x_gens[random_gene_index][interval_number]
            print(f"Мутировал ген c индексом {random_gene_index} с {random_gene} на {mutated_gene} (значение {value})", end='\n')
            if value == -1:
                print(f"Мутация №{_ + 1} не удалась, ген мутировал в недопустимое значение", end='\n')
                print(f"Позиция в исходной матрице: {random_gene_index} {interval_number}", end='\n')
                print(f"Исходная матрица:")
                self.print_matrix()
            if value != -1:
                self.y_gens[random_gene_index] = mutated_gene
                self.update_pheno()
                success = True
                break

        if not success:
            self.y_gens[random_gene_index] = original_gene
            print("Мутация не удалась после 3 попыток, ген остался исходным", end='\n')
    else:
        print("Мутация не произошла", end='\n')

  def crossover(self, ind: 'Individual'):
      if self.cross_prop[random.randint(0, 99)] == 1:
        first_child = Individual(ind.n, ind.x_gens, ind.full_x_gens, ind.p_mut, ind.p_cross, num=self.num, model=self.model)
        second_child = Individual(ind.n, ind.x_gens, ind.full_x_gens, ind.p_mut, ind.p_cross, num=self.num, model=self.model)
        
        if self.model == 1:
            split_dot = np.random.randint(1, len(self.y_gens) - 1)
            first_child.y_gens = self.y_gens[:split_dot] + ind.y_gens[split_dot:]
            second_child.y_gens = ind.y_gens[:split_dot] + self.y_gens[split_dot:]
        else:
            print("По модели Холланда")

            dots = [np.random.randint(1, len(self.y_gens) - 1), np.random.randint(1, len(self.y_gens) - 1)]
            left_split_dot = min(dots)
            right_split_dot = max(dots)
            
            first_child.y_gens = self.y_gens[:left_split_dot] + ind.y_gens[left_split_dot:right_split_dot] + self.y_gens[right_split_dot:]
            second_child.y_gens = ind.y_gens[:left_split_dot] + self.y_gens[left_split_dot:right_split_dot] + ind.y_gens[right_split_dot:]

        print(f"Прошёл кроссовер (его шанс был {self.p_cross})", end='\n')
        
        childs = [first_child, second_child]
        return childs
      else:
        print("Кроссовер не произошёл", end='\n')
        return None

In [21]:
class Evolution:
    def __init__(self, n, task_vector, p_mut, p_cross, population_size = 5, k = 3, model=1) -> None:
        self.population_size = population_size
        task_matrix = self.get_x_gens(np.array(task_vector), n)
        self.population = [Individual(n, task_vector, task_matrix, p_mut, p_cross, i+1, model) for i in range(self.population_size)]
        self.num_generations = k
        if model == 1:
            self.generation_buffer = {}
        else:
            self.generation_buffer = []
        self.model = model
        
    def get_x_gens(self, task_matrix, n):
        task_matrix_t = task_matrix.reshape(-1, 1)
        repeated_matrix = np.tile(task_matrix_t, (1, n))

        rows, cols = repeated_matrix.shape
        for i in range(rows):
            indices = np.arange(cols)
            np.random.shuffle(indices)
            num_neg_ones = np.random.randint(1, cols)
            
            for j in range(num_neg_ones):
                if repeated_matrix[i, indices[j]] != -1:
                    repeated_matrix[i, indices[j]] = -1

        return repeated_matrix
        
    def get_random_individual(self, current_ind):
        while True:
            random_ind = random.choice(self.population)
            if random_ind != current_ind:
                return random_ind
    def print_target_func(self, inds):
        inds_num = [ind.num for ind in inds]
        targets = [sum(ind.phenotype['max_feno']) for ind in inds]
        print("\t" + "Номер особи:" + " ".join(map(str, inds_num)))
        print("\t" + "Целевая функция:" + " ".join(map(str, targets)))

    def print_population(self):
        for ind in self.population:
            print(f"Особь {self.population.index(ind) + 1}")
            ind.print_info()
            
    def get_childs(self, ind1, ind2):
        childs = ind1.crossover(ind2)
        if childs is not None:
            for ind in childs:
                ind.mutation()
                ind.update_pheno()
                ind.isChild = True
                
            ind1.update_pheno()
            childs.append(ind1)
            return childs
        else:
            print("Идёт мутация:")
            mut_ind = Individual(ind1.n, ind1.x_gens, ind1.full_x_gens, ind1.p_mut, ind1.p_cross, num=ind1.num, model=ind1.model)
            mut_ind.y_gens = ind1.y_gens.copy()
            mut_ind.mutation()
            mut_ind.update_pheno()
            
            if sum(mut_ind.phenotype["max_feno"]) < sum(ind1.phenotype["max_feno"]):
                print("Фенотип изменился в лучшую сторону")
                print("Было:", ind1.phenotype["max_feno"], sum(ind1.phenotype["max_feno"]))
                print("Стало:", mut_ind.phenotype["max_feno"], sum(mut_ind.phenotype["max_feno"]))
            
            return [ind1, mut_ind]
        
    def get_best_individual(self, individuals):
        return min(individuals, key=lambda x: sum(x.phenotype['max_feno']))

    def get_top_in_generation(self, individuals):
        result = sorted(individuals, key=lambda x: sum(x.phenotype['max_feno']))
        print(f"Отсортированные особи (всего {len(result)}): ")
        return result[:self.population_size]

In [22]:
params = get_params()
params

(10, [18, 16, 14, 14, 17, 12, 19, 14, 19, 11], 1.0, 0.0, 10, 10, 1)

In [23]:
evolution = Evolution(*params)
#evolution = Evolution(7, task_matrix, 0.5, 0.5, 5, model=1)
ostanova_counter = 0
last_pheno = 0
gen_counter = 0


print("Начальное распределение:")
evolution.print_population()
    
generation = 0
while True:
    print(f"Формирование нового {generation + 1}-го поколения: \n")
    print("Исходная матрица:")
    evolution.population[0].print_matrix()
    
    for current_ind in evolution.population:
        partner_ind = evolution.get_random_individual(current_ind)

        print(f"Скрещивание особей {current_ind.num} и {partner_ind.num}")
        print(f"Особь {current_ind.num}")
        current_ind.print_info()

        print(f"Особь {partner_ind.num}")
        partner_ind.print_info()

        childs = evolution.get_childs(current_ind, partner_ind)
        [child.update_pheno() for child in childs]
        best_ind = evolution.get_best_individual(childs)
        

        if evolution.model == 1:
            print(f"Лучшая особь среди потомков:")
            best_ind.print_info()
            evolution.generation_buffer[current_ind.num] = best_ind
        else:
            print(f"Получены потомки:")
            current_ind.isChild = False
            best_ind.isChild = True
            current_ind.print_info()
            best_ind.print_info()
            evolution.generation_buffer.extend([best_ind])

    print(f"Поколение {generation + 1} сформировано!")
    if evolution.model == 1:
        print(f"Лучшая особь текущего поколения:")
        best_in_gen = evolution.get_best_individual(evolution.generation_buffer.values())
        best_in_gen.print_info()
        evolution.population = list(evolution.generation_buffer.values())
    else:
        all_inds = evolution.generation_buffer
        all_inds.extend(evolution.population)
        bests_in_gen = evolution.get_top_in_generation(all_inds)
        best_in_gen = bests_in_gen[0]
        print(f"Лучшая особь из лучших:", end = "\n")
        best_in_gen.print_info()
        print(f"Все особи поколения:", end = "\n")
        [ind.print_min_info() for ind in evolution.generation_buffer]
        print(f"Лучшие особи {generation + 1} поколения:", end="\n")
        
        print("Родители и их целевые функции:", end='\n')
        evolution.print_target_func(evolution.population)
        
        print("Дети и их целевые функции:", end='\n')
        evolution.print_target_func(evolution.generation_buffer)
        
        print("Лучшие особи и их целевые функции:", end='\n')
        evolution.print_target_func(bests_in_gen)
        
        [best_child.print_min_info() for best_child in bests_in_gen]
        [best_child.mark_as_parent() for best_child in bests_in_gen]
        evolution.population = bests_in_gen
        evolution.generation_buffer = []
        
        

    if last_pheno == sum(best_in_gen.phenotype['max_feno']):
        ostanova_counter += 1
    else:
        last_pheno = sum(best_in_gen.phenotype['max_feno'])
        ostanova_counter = 0

    gen_counter += 1
    print(f"Количество поколений без изменений: {ostanova_counter + 1}")

    if ostanova_counter >= evolution.num_generations - 1:
        print("Количество поколений без изменений превысило заданное значение. Обучение завершено!")
        print("Прошло полных поколений: ", gen_counter)
        break
    
    generation += 1

print("Обучение завершено!")

Начальное распределение:
Особь 1
Генотип -->
	18 16 14 14 17 12 19 14 19 11
	121 10 248 21 47 159 120 161 166 91
Фенотип:
1) 0...30 : [16, 14] | Сумма: 30 
2) 31...55 : [17] | Сумма: 17 
3) 56...80 : [] | Сумма: 0 
4) 81...105 : [11] | Сумма: 11 
5) 106...130 : [18, 19] | Сумма: 37 
6) 131...155 : [] | Сумма: 0 
7) 156...180 : [12, 14, 19] | Сумма: 45 <-- MAX
8) 181...205 : [] | Сумма: 0 
9) 206...230 : [] | Сумма: 0 
10) 231...255 : [14] | Сумма: 14 


Особь 2
Генотип -->
	18 16 14 14 17 12 19 14 19 11
	35 169 197 161 97 21 135 122 96 212
Фенотип:
1) 0...30 : [12] | Сумма: 12 
2) 31...55 : [18] | Сумма: 18 
3) 56...80 : [] | Сумма: 0 
4) 81...105 : [17, 19] | Сумма: 36 <-- MAX
5) 106...130 : [14] | Сумма: 14 
6) 131...155 : [19] | Сумма: 19 
7) 156...180 : [16, 14] | Сумма: 30 
8) 181...205 : [14] | Сумма: 14 
9) 206...230 : [11] | Сумма: 11 
10) 231...255 : [] | Сумма: 0 


Особь 3
Генотип -->
	18 16 14 14 17 12 19 14 19 11
	159 191 202 151 97 188 89 149 208 110
Фенотип:
1) 0...30 : 